In [ ]:
using System.Threading.Tasks;

static class SequentionalScheduler
{
    private static Task _prevTask = Task.CompletedTask;

    private static readonly object GlobalLocker = new object();

    public static Task Run(Action job)
    {
        lock (GlobalLocker)
        {
            return _prevTask = _prevTask.ContinueWith(t => job());
        }
    }

    public static Task Run(Func<Task> job)
    {
        lock (GlobalLocker)
        {
            return _prevTask = _prevTask.ContinueWith(t => job());
        }
    }
}

In [ ]:
SequentionalScheduler.Run(() => "Hello, world!".Display());
await Task.Delay(TimeSpan.FromMilliseconds(200));

Hello, world!

In [ ]:
Enumerable.Range(1, 20).AsParallel().ForAll(i => {
    SequentionalScheduler.Run(async () => {
        await Task.Delay(TimeSpan.FromSeconds(2));
        $"performing task {i}".Display();
    });
});

await Task.Delay(TimeSpan.FromSeconds(10));

performing task 16

performing task 20

performing task 3

performing task 11

performing task 12

...

performing task 13

performing task 14

performing task 17

performing task 1

performing task 5

In [ ]:
int Sum(int n) => Enumerable.Range(1, n).Sum();

void PrintSum(int n) => SequentionalScheduler.Run(() => $"sum {n:N0}: {Sum(n):N0}".Display());
PrintSum(65_535);
PrintSum(65_536);

await Task.Delay(TimeSpan.FromSeconds(0.5));

sum 65 535: 2 147 450 880

In [ ]:
IEnumerable<ulong> Range (ulong n) { for (var i = 0UL; i < n; ++i) yield return i; }
decimal Sum(ulong n) => Range(n).Sum(i => (decimal)i);

void PrintSum(ulong n) => SequentionalScheduler.Run(() => $"sum {n:N0}: {Sum(n):N0}".Display());
PrintSum(65_536L);
PrintSum(100_000_000L);

ulong Pow2(uint n) => (ulong)Math.Pow(2, n);
void Sum(uint n) => Sum(Pow2(n));
void PrintSum(uint n) => PrintSum(Pow2(n));
PrintSum(27u);

await Task.Delay(TimeSpan.FromSeconds(7));

sum 65 536: 2 147 450 880

sum 100 000 000: 4 999 999 950 000 000

sum 134 217 728: 9 007 199 187 632 128

In [ ]:
void RunVanilla(int start, int count)
{
    Enumerable.Range(start, count).AsParallel().ForAll(i => {
        var n = Pow2((uint)i);
        $"{i}: sum {n:N0} = {Sum(n):N0}".Display();
    });
}

async Task RunWithSequentionalScheduler(int start, int count)
{
    var tasks = new List<Task>();

    Enumerable.Range(start, count).AsParallel().ForAll(i => {
        var n = Pow2((uint)i);
        tasks.Add(SequentionalScheduler.Run(() => $"{i}: sum {n:N0} = {Sum(n):N0}".Display()));
    });

    "tasks scheduled".Display();

    await Task.WhenAll(tasks);
}

In [ ]:
RunVanilla(21, 9);

21: sum 2 097 152 = 2 199 022 206 976

22: sum 4 194 304 = 8 796 090 925 056

23: sum 8 388 608 = 35 184 367 894 528

24: sum 16 777 216 = 140 737 479 966 720

25: sum 33 554 432 = 562 949 936 644 096

26: sum 67 108 864 = 2 251 799 780 130 816

27: sum 134 217 728 = 9 007 199 187 632 128

28: sum 268 435 456 = 36 028 796 884 746 240

29: sum 536 870 912 = 144 115 187 807 420 416

In [ ]:
await RunWithSequentionalScheduler(21, 9);

tasks scheduled

21: sum 2 097 152 = 2 199 022 206 976

22: sum 4 194 304 = 8 796 090 925 056

25: sum 33 554 432 = 562 949 936 644 096

23: sum 8 388 608 = 35 184 367 894 528

26: sum 67 108 864 = 2 251 799 780 130 816

24: sum 16 777 216 = 140 737 479 966 720

27: sum 134 217 728 = 9 007 199 187 632 128

29: sum 536 870 912 = 144 115 187 807 420 416

28: sum 268 435 456 = 36 028 796 884 746 240

In [ ]:
RunVanilla(29, 1);

29: sum 536 870 912 = 144 115 187 807 420 416

In [ ]:
await RunWithSequentionalScheduler(29, 1);

tasks scheduled

29: sum 536 870 912 = 144 115 187 807 420 416